In [1]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
train_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='val', transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ../data/imagenet-mini.

In [5]:
model_path = '../models/model_CNN_scripted.pt'
model = torch.jit.load(model_path, map_location=device).to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [7]:
num_epochs = 50

In [8]:
log_df = pd.DataFrame(columns=['epoch', 'accuracy', 'val_loss'])

In [9]:
for epoch in tqdm(range(num_epochs), desc="Total Training Progress"):
    model.train()
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy:.2f}%, Validation Loss: {val_loss/len(test_loader):.4f}')

    log_df = log_df.append({'epoch': epoch+1, 'accuracy': accuracy, 'val_loss': val_loss/len(test_loader)}, ignore_index=True)

log_df.to_csv('training_log.csv', index=False)


scripted_trained_model = torch.jit.script(model)
scripted_trained_model.save("trained_models/trained_model_scripted_Modified_CNN.pt")

Total Training Progress:   0%|          | 0/5 [00:18<?, ?it/s]A


KeyboardInterrupt: 